#### Load Unsloath

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 32768 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
load_in_8bit = True

/home/novil/venvs/mistral/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Mistral patching release 2024.6
   \\   /|    GPU: NVIDIA RTX 6000 Ada Generation. Max memory: 47.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: Will load unsloth/mistral-7b-v0.3-bnb-4bit as a legacy tokenizer.


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32, # generally set to 2*rank(r)
    lora_dropout = 0.05, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


#### Prepare dataset

In [4]:
import json
with open('./datasets/Electrical-engineering.json', 'r') as fp:
    data = json.load(fp)


In [5]:
data[0]

{'instruction': 'You are an electrical engineer and you will answer questions related to electrical engineering.',
 'input': "What is Ohm's law?",
 'output': "Ohm's law states that the electric current through a conductor between two points is directly proportional to the voltage across the two points. It is represented by the equation V = RI, where V is the voltage, R is the resistance, and I is the current."}

In [6]:
elec_eng_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = elec_eng_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass




In [7]:
from datasets import load_dataset, Dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset =Dataset.from_list(data)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 1131/1131 [00:00<00:00, 149109.13 examples/s]


#### Train the model

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


In [9]:

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 1,
        # max_steps = 60, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        run_name="mistral_01"
    ),
)

Map (num_proc=2): 100%|██████████| 1131/1131 [00:00<00:00, 1720.42 examples/s]


#### `Show current memory stats`

In [10]:
##### 
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX 6000 Ada Generation. Max memory = 47.988 GB.
4.992 GB of memory reserved.


In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,131 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 70
 "-____-"     Number of trainable parameters = 167,772,160
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: novilsaikumar-a (al_finetune). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.888100
2,1.799600
3,1.590000
4,1.278100
5,1.066300
6,0.925200
7,0.811100
8,0.852000
9,0.758700
10,0.662400


In [ ]:
#### wandb : d8e72282674ac1460f84fe4a77ef6cc18591451f

In [12]:
### used memory 

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

249.5962 seconds used for training.
4.16 minutes used for training.
Peak reserved memory = 6.652 GB.
Peak reserved memory for training = 1.66 GB.
Peak reserved memory % of max memory = 13.862 %.
Peak reserved memory for training % of max memory = 3.459 %.


##### Inference

In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [33]:

inputs = tokenizer(
[
    elec_eng_prompt.format(
        "answer the following questions", # instruction
        "explain Kirchhoff law in 5 lines", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
response = tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [34]:
response[0].split('\n\n')[-1]

"### Response:\nKirchhoff's law states that the sum of currents entering a junction in an electrical circuit must equal the sum of currents leaving that junction. It is a fundamental principle in circuit analysis and design.</s>"

In [16]:
"""

['<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe next numbers in the Fibonacci sequence are 13, 21, 34, 55, 89, 144, and so on.</s>']

"### Response:\nOhm's law states that the current through a conductor between two points is directly proportional to the voltage across the two points. It is represented by the equation V = RI, where V is the voltage, R is the resistance, and I is the current.</s>"


Explain electric circuit and it's laws
"### Response:\nAn electric circuit is a network of conductors and components that allow the flow of electric current. It follows the laws of electricity, including Ohm's Law, which states that the current through a conductor between two points is directly proportional to the voltage across the two points.</s>"

what are the laws of electricity?
"### Response:\nThe laws of electricity include Ohm's Law, which states that the current through a conductor between two points is directly proportional to the voltage across the two points. Kirchhoff's Laws describe the conservation of current and energy in electrical circuits.</s>"


"### Response:\nKirchhoff's law states that the sum of currents entering a junction in an electrical circuit must equal the sum of currents leaving that junction. It is a fundamental principle in circuit analysis and design.</s>"


"""

tensor([[    1, 21579,  1117,  1164, 13894,  1137, 14734,  1032,  4406, 29493,
          6649,  2128,  1163,  1164,  3555,  1137,  6080,  4397,  3526, 29491,
         12786,  1032,  3667,  1137,  7350,  2767,  3459,  1042,  1040,  2927,
         29491,   781,   781, 28100,  3901,  3880, 29515,   781, 23378,  1209,
          1040, 16950,  1034, 29479, 13882,  8536, 29491,   781,   781, 28100,
         12000, 29515,   781, 29508, 29493, 29473, 29508, 29493, 29473, 29518,
         29493, 29473, 29538, 29493, 29473, 29550, 29493, 29473, 29551,   781,
           781, 28100, 12875, 29515,   781,  1782,  2447,  6319,  1065,  1040,
          1169,  1361,  1034, 28895,  8536,  1228, 29473, 29508, 29538, 29493,
         29473, 29518, 29508, 29493, 29473, 29538, 29549, 29493, 29473, 29550,
         29550, 29493, 29473, 29551, 29542, 29493, 29473, 29508, 29549, 29549,
         29493,  1072,  1347,  1124, 29491,     2]], device='cuda:0')

In [23]:
response

["<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nanswer the following questions\n\n### Input:\nExplain ohm's law\n\n### Response:\nOhm's law states that the current through a conductor between two points is directly proportional to the voltage across the two points. It is represented by the equation V = RI, where V is the voltage, R is the resistance, and I is the current.</s>"]

In [26]:
data

[{'instruction': 'You are an electrical engineer and you will answer questions related to electrical engineering.',
  'input': "What is Ohm's law?",
  'output': "Ohm's law states that the electric current through a conductor between two points is directly proportional to the voltage across the two points. It is represented by the equation V = RI, where V is the voltage, R is the resistance, and I is the current."},
 {'instruction': 'You are an electrical engineer and you will answer questions related to electrical engineering.',
  'input': 'How does a transformer work?',
  'output': 'A transformer works on the principle of electromagnetic induction and is used to change the voltage level of alternating current (AC). It consists of two coils: the primary and secondary, which are not electrically connected but linked by a magnetic field. When AC flows through the primary coil, it creates a varying magnetic field, inducing a voltage in the secondary coil. The voltage change between the pr

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-13b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA RTX 6000 Ada Generation. Max memory: 47.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/llama-2-13b-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 8.0, it can be magically be extended to 32768!


In [4]:
FastLanguageModel.for_inference(model)

In [7]:
inputs = tokenizer(
[
    elec_eng_prompt.format(
        "answer the following questions", # instruction
        "explain Kirchhoff law in 5 lines", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 4096, use_cache = True)
response = tokenizer.batch_decode(outputs)

KeyboardInterrupt: 

In [ ]:
response